In [ ]:
import os
import netCDF4 as nc
import numpy as np
import tqdm


sigma = 5.67e-8  # W/(m^2·K^4)


####需要输入的值有哪些？请输入文件路径：
path_LWDR = r'Y:\MODIS_interpret\reBuild\2022'
path_LWUR = r'Z:\ws\LessRad_LWUR\2022'
OUT_PATH=r'Z:\ws\LessRad_LWNR\2022'

os.chdir(path_LWDR)
file_chdir = os.getcwd()
nc_name1 = [] #csv数据名称（含有.csv扩展名）
for root, dirname, files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.nc':
            nc_name1.append(path_LWDR + '\\' + file)
            
            

os.chdir(path_LWUR)
file_chdir = os.getcwd()
nc_name3 = [] #csv数据名称（含有.csv扩展名）
for root, dirname, files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.nc':
            nc_name3.append(path_LWUR + '\\' + file)   



N = len(nc_name1)
#第一个循环，时间间隔为1，
for q in tqdm.trange(0,N):
    #####将每天的24个数据一起处理，为了水汽和温度取平均值之后的计算方便。
    filename1 = nc_name1[q]
    with nc.Dataset(filename1) as file1:
        file1.set_auto_mask(False)  # 可选
        variables_1 = {x: file1[x][()] for x in file1.variables}
    A = variables_1['LessRad_LWDR']
            #####注意 看对应年份经度范围是否需要改变。修正：其实是数据下载的格式问题比如2017年不需要，但2009,2015年就需要
    LWDR = A
    STORE_LWDR = []
    for k in range(0,24,1):
        T = LWDR[k,:,:]
        STORE_LWDR.append(T)
    NSTORE_LWDR = np.array(STORE_LWDR)
    filename3 = nc_name3[q]
    with nc.Dataset(filename3) as file1:
        file1.set_auto_mask(False)  # 可选
        variables_1 = {x: file1[x][()] for x in file1.variables}
    A = variables_1['lwur']
        #####注意 看对应年份经度范围是否需要改变。修正：其实是数据下载的格式问题比如2017年不需要，但2009,2015年就需要
    LWUR = A  
    STORE_LWUR = []
    for k in range(0,24,1):
            T = LWUR[k][:][:]
            STORE_LWUR.append(T)
    NSTORE_LWUR = np.array(STORE_LWUR)
    predicted = NSTORE_LWDR -  NSTORE_LWUR

    # 将predicted变为三维
    predicted_3d = predicted.reshape((24, 3600, 7200))




    date =filename3[-24:-16]
    output_file = os.path.join(OUT_PATH, f'LessRad_LWNR_{date}_5km_1hour_v1.nc')
    with nc.Dataset(output_file, 'w', format='NETCDF4') as f:
        # 创建维度和变量
        f.createDimension('time', 24)
        f.createDimension('latitude', 3600)
        f.createDimension('longitude', 7200)
        time_var = f.createVariable('time', 'i4', ('time',))
        lat_var = f.createVariable('latitude', 'f4', ('latitude',))
        lon_var = f.createVariable('longitude', 'f4', ('longitude',))
        lwdr_var = f.createVariable('lwnr', np.int16, 
                                           ('time', 'latitude', 'longitude'), 
                                           fill_value=-999, zlib=True, complevel=9)

        
        # 设置变量属性
        time_var.units = 'hours since {}-{}-{} 00:00:00'.format(filename3[-24:-20], filename3[-20:-18], filename3[-18:-16])
        time_var.calendar = 'gregorian'
        lat_var.units = 'degrees_north'
        lon_var.units = 'degrees_east'
        lwdr_var.units = 'W/m^2'
        lwdr_var.long_name = "longwave net radiation"
        lwdr_var.scale_factor = 0.1
        # 写入数据
        time = np.arange(24)
        lon = np.linspace(-180, 180, 7200)
        lat = np.linspace(90, -90, 3600)
        time_var[:] = time
        lat_var[:] = lat
        lon_var[:] = lon 
        lwdr_var[:] = predicted_3d
    print(f"数据已保存到文件 {output_file} ")

        
        
        
